# Requirements

In [8]:
import os
import json
import pandas as pd
import numpy as np
import json
import pickle
import random
import unicodedata
import requests
import re
import pyconll # universal dependencies parser
import spacy
import io

import sddk

In [9]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [10]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_preprocessed_20201127.json", "df", conf)
# save locally for the next time
#AGT.to_json("../data/large_files/AGT.json")

In [11]:
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,sentences,n_sentences
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...","[ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν ...",3252
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,[Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,[Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβά...,10245
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...","[Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα, ...",1982
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,[τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ...,152


# Spacy NLP model

In [17]:
#ag_lemma_lookup = nlp.vocab.lookups.get_table("lemma_lookup")
#ag_lemma_lookup_merged = nlp.vocab.lookups.get_table("lemma_lookup_merged")

In [12]:
nlp = spacy.load("../models/spacy_grc_model_6")
from lemmatization import *

In [14]:
nlp.remove_pipe("grc_doc_lemmatizer")

('grc_doc_lemmatizer', <function lemmatization.grc_doc_lemmatizer(doc)>)

In [15]:
nlp.add_pipe(grc_doc_lemmatizer, "grc_doc_lemmatizer", after="tagger")

In [16]:
test_sentence = AGT[AGT["author_id"]=="tlg0031paul"].iloc[3]["sentences"][0]
test_sentence

'ΠΑΥΛΟΣ ἀπόστολος, οὐκ ἀπʼ ἀνθρώπων οὐδὲ διʼ ἀνθρώπου ἀλλὰ διὰ Ἰησοῦ Χριστοῦ καὶ θεοῦ πατρὸς τοῦ ἐγείραντος αὐτὸν ἐκ νεκρῶν, καὶ οἱ σὺν ἐμοὶ πάντες ἀδελφοί, ταῖς ἐκκλησίαις τῆς Γαλατίας·'

In [18]:
doc = nlp(test_sentence)
print([(token.text, token.pos_, token.lemma_) for token in doc])

[('ΠΑΥΛΟΣ', 'VERB', 'Παῦλος'), ('ἀπόστολος', 'NOUN', 'ἀπόστολος'), (',', 'PUNCT', ','), ('οὐκ', 'ADV', 'οὐ'), ('ἀπʼ', 'ADP', 'ἀπό'), ('ἀνθρώπων', 'NOUN', 'ἄνθρωπος'), ('οὐδὲ', 'CCONJ', 'οὐδέ'), ('διʼ', 'ADP', 'διά'), ('ἀνθρώπου', 'NOUN', 'ἄνθρωπος'), ('ἀλλὰ', 'CCONJ', 'ἀλλά'), ('διὰ', 'ADP', 'διά'), ('Ἰησοῦ', 'PROPN', 'Ἰησοῦς'), ('Χριστοῦ', 'PROPN', 'Χριστός'), ('καὶ', 'CCONJ', 'καί'), ('θεοῦ', 'NOUN', 'θεός'), ('πατρὸς', 'NOUN', 'πατήρ'), ('τοῦ', 'DET', 'ὁ'), ('ἐγείραντος', 'VERB', 'ἐγείρω'), ('αὐτὸν', 'PRON', 'αὐτός'), ('ἐκ', 'ADP', 'ἐκ'), ('νεκρῶν', 'NOUN', 'νεκρών'), (',', 'PUNCT', ','), ('καὶ', 'CCONJ', 'καί'), ('οἱ', 'DET', 'ὁ'), ('σὺν', 'ADP', 'σύν'), ('ἐμοὶ', 'PRON', 'ἐγώ'), ('πάντες', 'ADJ', 'πᾶς'), ('ἀδελφοί', 'NOUN', 'ἀδελφός'), (',', 'PUNCT', ','), ('ταῖς', 'DET', 'ὁ'), ('ἐκκλησίαις', 'NOUN', 'ἐκκλησία'), ('τῆς', 'DET', 'ὁ'), ('Γαλατίας', 'NOUN', 'Γαλατία'), ('·', 'PUNCT', '·')]


In [19]:
# test whether our lookup table work
#ag_lemma_lookup = nlp.vocab.lookups.get_table("lemma_lookup")
ag_lemma_lookup["VERB"]["μνήσομαι"]

'μιμνήσκω'

In [12]:
def apply_nlp(sentences_list):
    spacy_docs = []
    for sentence in sentences_list:
        doc = nlp(sentence)
        spacy_docs.append(doc)
    return spacy_docs

In [13]:
paul_test = apply_nlp(AGT[AGT["author_id"]=="tlg0031paul"].iloc[3]["sentences"])
for doc in paul_test[:10]:
    print([(token.text, token.pos_, token.lemma_) for token in doc])

[('ΠΑΥΛΟΣ', 'VERB', 'Παῦλος'), ('ἀπόστολος', 'NOUN', 'ἀπόστολος'), (',', 'PUNCT', ','), ('οὐκ', 'ADV', 'οὐ'), ('ἀπʼ', 'ADP', 'ἀπό'), ('ἀνθρώπων', 'NOUN', 'ἄνθρωπος'), ('οὐδὲ', 'CCONJ', 'οὐδέ'), ('διʼ', 'ADP', 'διά'), ('ἀνθρώπου', 'NOUN', 'ἄνθρωπος'), ('ἀλλὰ', 'CCONJ', 'ἀλλά'), ('διὰ', 'ADP', 'διά'), ('Ἰησοῦ', 'PROPN', 'Ἰησοῦς'), ('Χριστοῦ', 'PROPN', 'Χριστός'), ('καὶ', 'CCONJ', 'καί'), ('θεοῦ', 'NOUN', 'θεός'), ('πατρὸς', 'NOUN', 'πατήρ'), ('τοῦ', 'DET', 'ὁ'), ('ἐγείραντος', 'VERB', 'ἐγείρω'), ('αὐτὸν', 'PRON', 'αὐτός'), ('ἐκ', 'ADP', 'ἐκ'), ('νεκρῶν', 'NOUN', 'νεκρών'), (',', 'PUNCT', ','), ('καὶ', 'CCONJ', 'καί'), ('οἱ', 'DET', 'ὁ'), ('σὺν', 'ADP', 'σύν'), ('ἐμοὶ', 'PRON', 'ἐγώ'), ('πάντες', 'ADJ', 'πᾶς'), ('ἀδελφοί', 'NOUN', 'ἀδελφός'), (',', 'PUNCT', ','), ('ταῖς', 'DET', 'ὁ'), ('ἐκκλησίαις', 'NOUN', 'ἐκκλησία'), ('τῆς', 'DET', 'ὁ'), ('Γαλατίας', 'NOUN', 'Γαλατία'), ('·', 'PUNCT', '·')]
[('χάρις', 'NOUN', 'χάρις'), ('ὑμῖν', 'PRON', 'ὑμεῖς'), ('καὶ', 'CCONJ', 'καί'), ('εἰρήνη', 'NOU

In [14]:
arist_test = apply_nlp(AGT[AGT["doc_id"]=="tlg0086.tlg010"]["sentences"].tolist()[0])
for doc in arist_test[:10]:
    print([(token.text, token.pos_, token.lemma_) for token in doc])

[('πᾶσα', 'DET', 'πᾶς'), ('τέχνη', 'NOUN', 'τέχνη'), ('καὶ', 'CCONJ', 'καί'), ('πᾶσα', 'DET', 'πᾶς'), ('μέθοδος', 'NOUN', 'μέθοδος'), (',', 'PUNCT', ','), ('ὁμοίως', 'ADV', 'ὅμοιος'), ('δὲ', 'PART', 'δέ'), ('πρᾶξίς', 'ADJ', 'πρᾶξις'), ('τε', 'PART', 'τε'), ('καὶ', 'CCONJ', 'καί'), ('προαίρεσις', 'NOUN', 'προαίρεσις'), (',', 'PUNCT', ','), ('ἀγαθοῦ', 'ADJ', 'ἀγαθός'), ('τινὸς', 'PRON', 'τίς'), ('ἐφίεσθαι', 'VERB', 'ἐφίημι'), ('δοκεῖ', 'VERB', 'δοκέω'), ('·', 'PUNCT', '·')]
[('διὸ', 'ADV', 'διό'), ('καλῶς', 'ADV', 'καλός'), ('ἀπεφήναντο', 'VERB', 'ἀποφαίνω'), ('τἀγαθόν', 'ADJ', 'ἀγαθός'), (',', 'PUNCT', ','), ('οὗ', 'PRON', 'ὅς'), ('πάντʼ', 'ADJ', 'πᾶς'), ('ἐφίεται', 'VERB', 'ἐφίημι'), ('.', 'PUNCT', '.')]
[('διαφορὰ', 'NOUN', 'διαφορά'), ('δέ', 'ADV', 'δέ'), ('τις', 'PRON', 'τις'), ('φαίνεται', 'VERB', 'φαίνω'), ('τῶν', 'DET', 'ὁ'), ('τελῶν', 'VERB', 'τελέω'), ('·', 'PUNCT', '·')]
[('τὰ', 'DET', 'ὁ'), ('μὲν', 'PART', 'μέν'), ('γάρ', 'PART', 'γάρ'), ('εἰσιν', 'VERB', 'εἰμί'), ('ἐνέργειαι

In [15]:
len(AGT)

1457

In [16]:
%%time
AGT_docs_series = AGT["sentences"].apply(apply_nlp)

CPU times: user 1h 25min 52s, sys: 17.3 s, total: 1h 26min 9s
Wall time: 1h 26min 11s


In [17]:
%%time
AGT_tagged = AGT_docs_series.apply(lambda docs: [[(token.text, token.pos_, token.lemma_) for token in doc] for doc in docs])

CPU times: user 55.5 s, sys: 2.09 s, total: 57.6 s
Wall time: 57.6 s


In [29]:
agt_tagged_df = pd.DataFrame(AGT_tagged)

In [21]:
sddk.write_file("SDAM_data/AGT/AGT_lemmatized_20201127.json", AGT, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/AGT_lemmatized_20201127.json"


In [20]:
# reading it back
agt_tagged_df = sddk.read_file("SDAM_data/AGT/AGT_tagged_20201127.json", "df", conf)

In [ ]:
"\W"

In [45]:
token = ["ἀρχόμενος ", "VERB", "ἄρχω"]

if (re.search("\W", token[2])):
    print("not valid word")
    token = [token[0], "PUNKT", token[2]]
print(token)

not valid word
['ἀρχόμενος ', 'PUNKT', 'ἄρχω']


In [ ]:
invalid_words =[] 
tagged_sentences
for sentence in tagged_sentences:
        for token in sentence:
            if (re.search("\W", token[2])):
                token = [token[0], "PUNKT", token[2]]
            new_sentence.append(token)
        new_sentences.append(new_sentence)

In [49]:
def invalid_words_to_punkt(tagged_sentences):
    new_sentences = []
    for sentence in tagged_sentences:
        new_sentence = []
        for token in sentence:
            if (re.search("\W", token[2])):
                token = [token[0], "PUNKT", token[2]]
            new_sentence.append(token)
        new_sentences.append(new_sentence)
    return new_sentences   
    
    
#agt_tagged_df["sentences"]apply.head(5)

In [57]:
agt_tagged_clean = agt_tagged_df["sentences"].apply(invalid_words_to_punkt)

In [58]:
def get_lemmatized_sentences(docs):
    tags = ["NOUN", "PROPN","VERB", "ADJ"]
    sents = []  
    for doc in docs:
        sent = []
        for token in doc:
            if token[1] in tags:
                sent.append(token[2])
        sents.append(sent)
    return sents

In [59]:
lemmatized_sentences = agt_tagged_df_clean.apply(get_lemmatized_sentences)

In [60]:
AGT["lemmatized_sentences"] = lemmatized_sentences

In [61]:
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,sentences,n_sentences,lemmatized_sentences
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...","[ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν ...",3252,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,[Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,[Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβά...,10245,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...","[Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα, ...",1982,"[[ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή, μελί..."
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,[τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ...,152,"[[ῥόδον, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλ..."


In [63]:
sddk.write_file("SDAM_data/AGT/AGT_lemmatized_20201201.json", AGT, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/AGT_lemmatized_20201201.json"


# Backup

In [ ]:
%%time
tagged_list = []
for work_sentences in AGT["sentences"]:
    docs = apply_nlp(work_sentences)
    tagged = [[(token.text, token.pos_, token.lemma_) for token in doc] for doc in docs]
    tagged_list.append(tagged)
sddk.write_file("SDAM_data/AGT/agt_tagged_list_0-499.json", tagged_list, conf)
tagged_list = []

In [186]:
%%time
tagged_list = []
for work_sentences in AGT["sentences"][:500]:
    docs = apply_nlp(work_sentences)
    tagged = [[(token.text, token.pos_, token.lemma_) for token in doc] for doc in docs]
    tagged_list.append(tagged)
sddk.write_file("SDAM_data/AGT/agt_tagged_list_0-499.json", tagged_list, conf)
tagged_list = []

Your <class 'list'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/tagged_list_0-499.json"
CPU times: user 26min 38s, sys: 17.4 s, total: 26min 56s
Wall time: 27min 5s


In [187]:
%%time
for work_sentences in AGT["sentences"][500:1000]:
    docs = apply_nlp(work_sentences)
    tagged = [[(token.text, token.pos_, token.lemma_) for token in doc] for doc in docs]
    tagged_list.append(tagged)
sddk.write_file("SDAM_data/AGT/agt_tagged_list_500-999.json", tagged_list, conf)
tagged_list = []

Your <class 'list'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/tagged_list_500-999.json"
CPU times: user 36min 10s, sys: 23 s, total: 36min 33s
Wall time: 36min 46s


In [ ]:
%%time
tagged_list = []
for work_sentences in AGT["sentences"][1000:]:
    docs = apply_nlp(work_sentences)
    tagged = [[(token.text, token.pos_, token.lemma_) for token in doc] for doc in docs]
    tagged_list.append(tagged)
sddk.write_file("SDAM_data/AGT/agt_tagged_list_1000-1199.json", tagged_list, conf)
tagged_list = []

In [3]:
%%time
tagged_lists = []
for url in ["SDAM_data/AGT/agt_tagged_list_0-499.json", "SDAM_data/AGT/agt_tagged_list_500-999.json", "SDAM_data/AGT/agt_tagged_list_1000-end.json"]:
    tagged_lists.extend(sddk.read_file(url, "list", conf))

CPU times: user 1min 43s, sys: 27.7 s, total: 2min 11s
Wall time: 4min 31s


In [5]:
tagged_lists[:2]

[[[['ἀρχόμενος', 'VERB', 'ἄρχω'],
   ['σέο', 'PRON', 'σύ'],
   [',', 'PUNCT', ','],
   ['Φοῖβε', 'NOUN', 'Φοῖβος'],
   [',', 'PUNCT', ','],
   ['παλαιγενέων', 'ADJ', 'παλαιγενεών'],
   ['κλέα', 'NOUN', 'κλέος'],
   ['φωτῶν', 'NOUN', 'φώς'],
   ['μνήσομαι', 'VERB', 'μιμνήσκω'],
   [',', 'PUNCT', ','],
   ['οἳ', 'PRON', 'ὅς'],
   ['Πόντοιο', 'NOUN', 'ποντοιό'],
   ['κατὰ', 'ADP', 'κατά'],
   ['στόμα', 'NOUN', 'στόμα'],
   ['καὶ', 'CCONJ', 'καί'],
   ['διὰ', 'ADP', 'διά'],
   ['πέτρας', 'NOUN', 'πέτρα'],
   ['Κυανέας', 'NOUN', 'Κυάνεαι'],
   ['βασιλῆος', 'NOUN', 'βασιλεύς'],
   ['ἐφημοσύνῃ', 'VERB', 'ἐφημοσυνῄ'],
   ['Πελίαο', 'NOUN', 'πελιαό'],
   ['χρύσειον', 'ADJ', 'χρύσεος'],
   ['μετὰ', 'ADP', 'μετά'],
   ['κῶας', 'NOUN', 'κῶας'],
   ['ἐύζυγον', 'VERB', 'ἐυζυγόν'],
   ['ἤλασαν', 'VERB', 'ἐλαύνω'],
   ['Ἀργώ', 'ADV', 'ἀργώ'],
   ['.', 'PUNCT', '.']],
  [['τοίην', 'NOUN', 'τοιήν'],
   ['γὰρ', 'PART', 'γάρ'],
   ['Πελίης', 'ADJ', 'πελιής'],
   ['φάτιν', 'NOUN', 'φάτις'],
   ['ἔκλυεν', '

In [6]:
def get_lemmatized_sentences(docs):
    tags = ["NOUN", "PROPN","VERB", "ADJ"]
    sents = []  
    for doc in docs:
        sent = []
        for token in doc:
            if token[1] in tags:
                sent.append(token[2])
        sents.append(sent)
    return sents

In [30]:
%%time
lemmatized_sentences = []
for tagged_list in tagged_lists_flat:
    lemmatized_sentences.append(get_lemmatized_sentences(tagged_list))

CPU times: user 9.94 s, sys: 3.2 s, total: 13.1 s
Wall time: 13.1 s


In [31]:
sddk.write_file("AGT_lemmatized_sentences.json", lemmatized_sentences, conf)

A file with the same name ("AGT_lemmatized_sentences.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'list'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/AGT_lemmatized_sentences.json"


In [32]:
len(lemmatized_sentences[0])

3252

In [33]:
AGT["lemmatized_sentences"] = lemmatized_sentences

In [37]:
AGT.columns

Index(['filename', 'author', 'title', 'wordcount', 'author_id', 'doc_id',
       'raw_date', 'date_avr', 'date_probs', 'date_manual', 'provenience',
       'clean_string', 'sentences', 'n_sentences', 'lemmatized_sentences',
       'tlg_epithet'],
      dtype='object')

In [38]:
AGT.drop(labels=["sentences", "clean_string"], axis=1, inplace=True)

In [39]:
sddk.write_file("SDAM_data/AGT/AGT_lemmatized_20201119.json", AGT, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/AGT_lemmatized_20201119.json"


# Do some additional cleaning & add lemmata column

In [4]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_lemmatized_20201119.json", "df", conf)

In [10]:
# additional cleaning of sentences
def clean_sentences(list_of_sentences):
    clean_sentences = []
    for sentence in list_of_sentences:
        clean_sentence = [re.sub("[\W\d_]", "", word) for word in sentence]
        clean_sentence = [word for word in clean_sentence if "ʼ" not in word]
        clean_sentence = [word for word in clean_sentence if word != ""]
        clean_sentences.append(clean_sentence)
    return clean_sentences

clean_sentences_series = AGT["lemmatized_sentences"].apply(clean_sentences)

AGT["lemmatized_sentences"] = clean_sentences_series

In [11]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
AGT["lemmata"] = AGT["lemmatized_sentences"].apply(flat_lemmata)

AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183, finally 14756899

14573055

In [12]:
sddk.write_file("SDAM_data/AGT/AGT_20201124.json", AGT, conf)

A file with the same name ("AGT_20201124.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/AGT_20201124.json"


# Experiments with serialization (not working!...)

In [6]:
from spacy.tokens import DocBin

def get_doc_bin_bytes(docs):
    """based on https://spacy.io/api/docbin"""
    doc_bin = DocBin()
    for doc in docs:
        doc_bin.add(doc)
    doc_bin_bytes = doc_bin.to_bytes()
    return doc_bin_bytes

In [30]:
# extract docs into a series of doc_bins
# save these doc_bins for later usage

#doc_bin_bytes_series = AGT["docs"].apply(get_doc_bin_bytes)
#doc_bin_bytes_series.to_pickle("../data/large_files/doc_bin_bytes_series_20201117.pickle")

In [4]:
# read the doc_bins obtained within previous run of the notebook
resp = conf[0].get(conf[1] + "SDAM_data/AGT/doc_bin_bytes_series_20201117.pickle")
resp

<Response [200]>

In [10]:
%%time
# read it back as a series
doc_bin_bytes_series = pd.read_pickle(io.BytesIO(resp.content))
# transform bytes into nlp objects
docs_bin_series = doc_bin_bytes_series.apply(lambda x: DocBin().from_bytes(x)) #.get_docs(nlp.vocab))

CPU times: user 12.2 s, sys: 2.11 s, total: 14.3 s
Wall time: 14.3 s


In [21]:
%%time
list_docs = []
for doc_bin in docs_bin_series:
    docs = list(doc_bin.get_docs(nlp.vocab))
    list_docs.append(docs)

CPU times: user 6min 33s, sys: 15.7 s, total: 6min 49s
Wall time: 6min 49s


In [11]:
list_docs

In [38]:
AGT["docs"] = list_docs
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,clean_string,sentences,n_sentences,docs
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...","[ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν ...",3252,"[(ἀρχόμενος, σέο, ,, Φοῖβε, ,, παλαιγενέων, κλ..."
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,[Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[(Θουκυδίδης, Ἀθηναῖος, ξυνέγραψε, τὸν, πόλεμο..."
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,[Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβά...,10245,"[(Τὸ, τῆς, φιλοσοφίας, ἔργον, ἔνιοί, φασιν, ἀπ..."
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...","[̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...",1982,"[(̔Αδύ, τι, τὸ, ψιθύρισμα, καὶ, ἁ, πίτυς, αἰπό..."
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,[τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ...,152,"[(τὰ, ῥόδα, τὰ, δροσόεντα, καὶ, ἁ, κατάπυκνος,..."


In [47]:
docs_0 = AGT.iloc[0]["docs"]

In [44]:
def get_lemmatized_sentences(docs):
    tags = ["NOUN", "VERB", "ADJ"]
    sents = []  
    for doc in docs:
        sent = []
        for token in doc:
            if token.pos_ in tags:
                sent.append(token.lemma_)
        sents.append(sent)
    return sents

In [45]:
AGT["lemmatized_sentences"] = AGT["docs"].apply(get_lemmatized_sentences)

In [41]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
AGT["lemmata"] = AGT["lemmatized_sentences"].apply(flat_lemmata)

In [ ]:
AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183

In [ ]:
# save locally
#AGT.drop("docs", axis=1).to_json("../data/large_files/AGT_20201117.json")

In [ ]:
#sddk.write_file("SDAM_data/AGT/AGT_20201117.json", AGT.drop("docs", axis=1, conf)

# Old stuff

In [17]:
%%time
AGT["lemmata"] = AGT.apply(lambda row: lemmatize_string(row["string"], all_lemmata=False, filter_by_postag=["n", "a", "v"], involve_unknown=True), axis=1)

CPU times: user 2min 26s, sys: 2.2 s, total: 2min 28s
Wall time: 2min 29s


In [18]:
all_lemmata = []
for work in AGT["lemmata"].tolist():
    all_lemmata.extend(work)

In [21]:
AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183

13812934

In [22]:
%%time
def lemmatize_sentences(sentences):
    lemmatized_sentences = []
    for sentence in sentences:
        lemmatized_sentence = lemmatize_string(sentence, all_lemmata=False, filter_by_postag=["n", "a", "v"], involve_unknown=True)
        lemmatized_sentences.append(lemmatized_sentence)
    return lemmatized_sentences

AGT["lemmatized_sentences"] = AGT["sentences"].apply(lemmatize_sentences)

CPU times: user 2min 46s, sys: 403 ms, total: 2min 47s
Wall time: 2min 47s


In [23]:
AGT["n_sentences"] = AGT['lemmatized_sentences'].apply(lambda x: len(x))

In [24]:
# look at some string
# e.g. paul's letter to the Galatians
AGT[AGT["author_id"]=="tlg0031paul"].iloc[3]["string"][:1000]

'\n\n                    ΠΑΥΛΟΣ ἀπόστολος, οὐκ ἀπʼ ἀνθρώπων οὐδὲ διʼ ἀνθρώπου\n\nἀλλὰ διὰ Ἰησοῦ Χριστοῦ καὶ θεοῦ πατρὸς τοῦ\n\nἐγείραντος αὐτὸν ἐκ νεκρῶν,\n                     \n                    καὶ οἱ σὺν ἐμοὶ πάντες\n\nἀδελφοί, ταῖς ἐκκλησίαις τῆς Γαλατίας·\n                     \n                    χάρις ὑμῖν καὶ\n\n\n\n εἰρήνη ἀπὸ θεοῦ πατρὸς ἡμῶν καὶ κυρίου Ἰησοῦ Χριστοῦ,\n                     \n                    τοῦ δόντος ἑαυτὸν ὑπὲρ τῶν ἁμαρτιῶν ἡμῶν ὅπως ἐξέληται\n\nἡμᾶς ἐκ τοῦ αἰῶνος τοῦ ἐνεστῶτος πονηροῦ κατὰ τὸ\n\nθέλημα τοῦ θεοῦ καὶ πατρὸς ἡμῶν,\n                     \n                    ᾧ ἡ δόξα εἰς τοὺς\n\nαἰῶνας τῶν αἰώνων· ἀμήν.\n                     \n\n                    Θαυμάζω ὅτι οὕτως ταχέως μετατίθεσθε ἀπὸ τοῦ καλέσαντος\n\nὑμᾶς ἐν χάριτι Χριστοῦ εἰς ἕτερον εὐαγγέλιον,\n                     \n                    ὃ\n\nοὐκ ἔστιν ἄλλο· εἰ μή τινές εἰσιν οἱ ταράσσοντες ὑμᾶς καὶ\n\nθέλοντες μεταστρέψαι τὸ εὐαγγέλιον τοῦ χριστοῦ.\n                     \n     

In [25]:
# and its lemmatized version
print(AGT[AGT["author_id"]=="tlg0031paul"].iloc[3]["lemmata"][:100])

['παυλος', 'ἀπόστολος', 'ἀπ', 'ἄνθρωπος', 'ἄνθρωπος', 'ἰησοῦ', 'χριστοῦ', 'θεάομαι', 'πατήρ', 'νεκρός', 'ἐμέω', 'πᾶς', 'ἀδελφός', 'ἐκκλησιάζω', 'γαλατίας', 'χάρις', 'εἰρήνη', 'θεάομαι', 'πατήρ', 'ἡμός', 'κύριος', 'ἰησοῦ', 'χριστοῦ', 'ἁμάρτημα', 'ἡμός', 'ἐξαιρέω', 'ἡμός', 'αἰών', 'πονηρός', 'θέλημα', 'θεάομαι', 'πατήρ', 'ἡμός', 'δοκέω', 'αἰών', 'αἰών', 'ἀμάω', 'θαυμάζω', 'μετατίθημι', 'χάρις', 'χριστοῦ', 'ἕτερος', 'εὐαγγέλιον', 'τίνω', 'εἶμι', 'μεταστρέφω', 'εὐαγγέλιον', 'χριστός', 'ἄγγελος', 'οὐρανός', 'εὐαγγελίζομαι', 'Πάρος', 'εὐηγγελισάμεθα', 'ἀνάθεμα', 'προερέω', 'λέγω', 'εἶμι', 'εὐαγγελίζομαι', 'Πάρος', 'παραλαμβάνω', 'ἀνάθεμα', 'ἄρτι', 'ἄνθρωπος', 'πειθός', 'θεός', 'ζάω', 'ἄνθρωπος', 'ἀρέσκω', 'ἄνθρωπος', 'ἀρέσκω', 'χριστοῦ', 'δοῦλος', 'γνωρίζω', 'ἀδελφός', 'εὐαγγέλιον', 'εὐαγγελίζομαι', 'ὑπ', 'ἐμέω', 'ἄνθρωπος', 'ἄνθρωπος', 'παραλαμβάνω', 'διδάσκω', 'ἀποκάλυψις', 'ἰησοῦ', 'χριστοῦ', 'ἠκούσατε', 'ἐμην', 'ἀναστροφή', 'ποτός', 'ἰουδαϊσμῷ', 'ἄατος', 'ὑπερβολην', 'διώκω', 'ἐκκλησιάζω

# Global corrections

# Corrections by means of GNT

In [44]:
GNT_df = pd.read_csv("https://raw.githubusercontent.com/biblicalhumanities/Nestle1904/master/morph/Nestle1904.csv", delim_whitespace=True, index_col=False, skiprows=1, names=("book", "ch_v", "func_morph", "pos1", "pos2", "strong", "lemma", "normalized"))
GNT_df.head(5)

,book,ch_v,func_morph,pos1,pos2,strong,lemma,normalized
0,Matt,1:1,Βίβλος,N-NSF,N-NSF,976,βίβλος,Βίβλος
1,Matt,1:1,γενέσεως,N-GSF,N-GSF,1078,γένεσις,γενέσεως
2,Matt,1:1,Ἰησοῦ,N-GSM,N-GSM,2424,Ἰησοῦς,Ἰησοῦ
3,Matt,1:1,Χριστοῦ,N-GSM,N-GSM,5547,Χριστός,Χριστοῦ
4,Matt,1:1,υἱοῦ,N-GSM,N-GSM,5207,υἱός,υἱοῦ


In [50]:
len(GNT_df["normalized"].tolist())

137779

In [49]:
len(GNT_df["normalized"].unique().tolist())

18444

In [46]:
GNT_morph_lemma =[(unicodedata.normalize("NFC", morph), unicodedata.normalize("NFC", lemma)) for morph, lemma in zip(GNT_df["func_morph"].tolist(), GNT_df["lemma"].tolist())]
GNT_morph_lemma[:10]

[('Βίβλος', 'βίβλος'),
 ('γενέσεως', 'γένεσις'),
 ('Ἰησοῦ', 'Ἰησοῦς'),
 ('Χριστοῦ', 'Χριστός'),
 ('υἱοῦ', 'υἱός'),
 ('Δαυεὶδ', 'Δαυίδ'),
 ('υἱοῦ', 'υἱός'),
 ('Ἀβραάμ.', 'Ἀβραάμ'),
 ('Ἀβραὰμ', 'Ἀβραάμ'),
 ('ἐγέννησεν', 'γεννάω')]

In [51]:
len(GNT_morph_lemma)

137779

In [53]:
GNT_morph_lemma_unique = list(set(GNT_morph_lemma))
len(GNT_morph_lemma_unique)

25258

In [ ]:
GNT_morph_lemma_unique

In [28]:
NT_df = AGT[AGT["author_id"].str.startswith("tlg0031")]
NT_lemmata = []
for lemmata in NT_df["lemmata"].tolist():
    NT_lemmata.extend(lemmata)

In [40]:
NT_freqdist = nltk.FreqDist(NT_lemmata).most_common()
NT_freqdist

[('πᾶς', 1086),
 ('ἁμός', 1078),
 ('εἶπον', 828),
 ('λέγω', 786),
 ('θεάομαι', 701),
 ('θεός', 620),
 ('ἡμός', 564),
 ('ἄνθρωπος', 522),
 ('ποιέω', 487),
 ('γίγνομαι', 482),
 ('σεύω', 476),
 ('ἔχω', 465),
 ('ἰησοῦς', 454),
 ('ἔρχομαι', 421),
 ('πατήρ', 397),
 ('εἶμι', 388),
 ('ἡμέρα', 380),
 ('υἱός', 371),
 ('κύριος', 371),
 ('εἶδον', 367),
 ('πνεῦμα', 357),
 ('πολύς', 341),
 ('λόγος', 329),
 ('ἰησοῦ', 325),
 ('δίδωμι', 312),
 ('ἀδελφός', 311),
 ('οὐρανός', 273),
 ('μαθητής', 262),
 ('ἐκεῖνος', 253),
 ('οἶδα', 249),
 ('ἀκούω', 247),
 ('γῆ', 246),
 ('μέγας', 239),
 ('λαλέω', 231),
 ('πίστις', 226),
 ('ἐμέω', 219),
 ('δοκέω', 217),
 ('χριστός', 214),
 ('ἀλλ', 214),
 ('ἅγιος', 203),
 ('ἵημι', 202),
 ('νόμος', 194),
 ('γυνή', 192),
 ('γιγνώσκω', 190),
 ('χριστοῦ', 182),
 ('δύναμαι', 176),
 ('πιστεύω', 174),
 ('ἐθέλω', 170),
 ('φωνέω', 165),
 ('βασίλεια', 162),
 ('ἔθνος', 162),
 ('ἔργον', 162),
 ('πρότερος', 161),
 ('χάρις', 158),
 ('ὁράω', 157),
 ('καρδία', 156),
 ('ζωή', 151),
 ('ἀνήρ', 1

[{'i': '432951',
  'f': 'ἰησοῦς',
  'b': 'ιησους',
  'l': 'Ἰησοῦς',
  'e': 'ιησους',
  'p': 'n-p---ma-',
  'd': '15766',
  's': 'Joshua',
  'a': 'attic'},
 {'i': '432956',
  'f': 'ἰησοῦς',
  'b': 'ιησους',
  'l': 'Ἰησοῦς',
  'e': 'ιησους',
  'p': 'n-s---mn-',
  'd': '15766',
  's': 'Joshua',
  'a': 'attic epic'}]

In [ ]:
"ἰησοῦ" : "ἰησοῦς",
"ἰησοῦν" : "ἰησοῦς",
"χριστοῦ" : "χριστός",
"χριστῷ" : "χριστός",
"κύριε" : "κύριος",
"κυρίου" : "κύριος",
"ἀβρααμ" : "ἀβραάμ"

In [41]:
[(tup[0], lemma_translator(tup[0]), tup[1]) for tup in NT_freqdist[:500]]

[('πᾶς', 'all, the whole', 1086),
 ('ἁμός', 'as, when / our, ours', 1078),
 ('εἶπον', 'to speak, say', 828),
 ('λέγω',
  'to gather, pick up / to say, speak / to lay asleep, lull to sleep',
  786),
 ('θεάομαι', 'to look on, gaze at, view, behold', 701),
 ('θεός', 'God', 620),
 ('ἡμός', '', 564),
 ('ἄνθρωπος', 'manfaced', 522),
 ('ποιέω', 'to make', 487),
 ('γίγνομαι', 'gi-geno', 482),
 ('σεύω', 'to put in quick motion: to drive, hunt, chase away', 476),
 ('ἔχω', 'to have', 465),
 ('ἰησοῦς', '', 454),
 ('ἔρχομαι', 'to come', 421),
 ('πατήρ', 'a father', 397),
 ('εἶμι', 'to go', 388),
 ('ἡμέρα', 'day', 380),
 ('υἱός', 'A son', 371),
 ('κύριος', 'having power / a lord, master', 371),
 ('εἶδον', 'to see', 367),
 ('πνεῦμα', 'a blowing', 357),
 ('πολύς', 'many', 341),
 ('λόγος', 'the word', 329),
 ('ἰησοῦ', '', 325),
 ('δίδωμι', 'to give', 312),
 ('ἀδελφός', 'sons of the same mother', 311),
 ('οὐρανός', 'heaven', 273),
 ('μαθητής', 'a learner, pupil', 262),
 ('ἐκεῖνος', 'the person there, th

In [26]:
# straightforward corrections might be added here

corrections = {
    "ἔχις" : "ἔχω"
}

for key in corrections.keys():
    AGT["lemmata"] = AGT["lemmata"].apply(lambda word_list: [corrections[key] if x == key else x for x in word_list])
    AGT["lemmatized_sentences"] = AGT["lemmatized_sentences"].apply(lambda sentences_list: [[corrections[key] if x == key else x for x in sentence] for sentence in sentences_list])

In [27]:
sddk.write_file("SDAM_data/AGT/AGT_20201110.json", AGT, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/AGT_20201110.json"
